# Washington DC Wards

In this notebook, I play around with the suggestions by [@labibrahman in this tweet](https://twitter.com/labibrahman/status/1350260884120092673).

You will need to install `geopandas` and `geoplot` (which come with `contextily`, if I understand correctly).

```shell
pip3 install matplotlib pandas geopandas geoplot contextily
```

Data here are provided by [Open Data DC](https://opendata.dc.gov/) and [ArcGIS.com](https://hub.arcgis.com/datasets/f5abc06dce4f4a03bde94d6f937996d6).

In [1]:
import geopandas
import geoplot

import matplotlib.pyplot as plt
import contextily as ctx
import pandas as pd

## Generate the datasets.

Here we join the ward-level data with a separate COVID-19 dataset (also from Open Data DC).

In [2]:
wards = geopandas.read_file("https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Administrative_Other_Boundaries_WebMercator/MapServer/31/query?outFields=*&where=1%3D1&f=geojson")
wards = wards.to_crs(epsg=3857)
covid_by_ward = geopandas.read_file("https://opendata.arcgis.com/datasets/94b5ab3b8b334d31aa395aea9bef2c10_27.geojson")
wards = wards.join(covid_by_ward.groupby("WARD").max(), on="NAME", lsuffix="_")

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1076)>

### Charts of data

We can chart the geo shapefiles with `geoplot`. I use `contextily` to paint in the faded background maps which I think add some useful geographic context.

In [ ]:
with plt.style.context("bmh"):
    plt.figure(figsize=(8, 8), dpi=150)
    ax = wards.plot(
        column=wards.MEDIAN_HH_INCOME.astype(int), 
        cmap="Greens", 
        legend=True,
        edgecolor='w', 
        legend_kwds={"fmt": "{:.0f}"},
        ax=plt.gca()
    )
    ax.axes.set_axis_off()
    ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerBackground, alpha=0.3)
    plt.title("Income")

In [ ]:
with plt.style.context("bmh"):
    plt.figure(figsize=(8, 8), dpi=150)
    ax = wards.plot(
        column=wards.PCT_BELOW_POV.astype(float), 
        cmap="Oranges", 
        legend=True,
        edgecolor='w',
        ax=plt.gca()
    )
    ax.axes.set_axis_off()
    ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerBackground, alpha=0.3)
    plt.title("Percent Below Poverty Line")

You can play some more with the `wards` dataframe, which is an enriched geopandas dataframe (i.e., not a vanilla pandas.DataFrame). If you want the regular pandas API, you can just run `pd.DataFrame(wards)` to get it back.

In [ ]:
with plt.style.context("ieee"):
    plt.figure(figsize=(4, 3), dpi=150, facecolor='white')
    plt.title("DC Ward Income vs Lives Lost due to COVID-19")
    plt.scatter(wards.MEDIAN_HH_INCOME.astype(int), wards.LIVES_LOST)
    plt.xlabel("Median household income (USD)")
    plt.ylabel("Lives lost")